In [7]:
import requests
import time
import json
from dotenv import load_dotenv
import os
from azure.eventhub import EventHubProducerClient, EventData

In [8]:
result = load_dotenv()

In [9]:
API_KEY = os.getenv('API_KEY')
EVENT_HUB_CONN_STR = os.getenv("EVENT_HUB_CONN_STR")
EVENT_HUB_NAME = os.getenv("EVENT_HUB_NAME")

In [10]:
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONN_STR,
    eventhub_name=EVENT_HUB_NAME
)

# Make API Call
def api_call(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"
    response = requests.get(url)
    # Ensure that request succeeds
    if response.status_code != 200:
        print(f"Error: Unable to fetch data for {city}. Status code: {response.status_code}")
        return None
    # Convert http request to dictionary type
    data = response.json()
    # Convert to json string
    data_json_str = json.dumps(data)
    return data_json_str

In [ ]:
# Call Open Weather API every 5 minutes and send to Azure Event Hubs
def api_call_timer(city1, city2, city3):
    cities = [city1, city2, city3]
    try:
        while True:
            event_data_batch = producer.create_batch()
            results = [api_call(city1), api_call(city2), api_call(city3)]
            for i in range(len(cities)):
                if results[i] is None:
                    print(f"Error: Failed to retrieve weather data for {cities[i]}")
                else:
                    event_data_batch.add(EventData(results[i]))
            producer.send_batch(event_data_batch)
            print(results)
            time.sleep(300)
    except KeyboardInterrupt:
        print("Stream stopped")
        print(results)
    finally:
        producer.close()
        print("Connection closed")

In [ ]:
api_call_timer("Miami", "Indianapolis", "Chicago")